In [36]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\jakan\anaconda3\lib\site-packages (0.0)


In [37]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [38]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [39]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [15]:
# Set features. This will also be used as your x values.
# Transit Properties
selected_features = df[['koi_disposition','koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [21]:
data = pd.get_dummies(selected_features)
data.head()

X = data[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol']]
y = data['koi_disposition_CONFIRMED'].values.reshape(-1,1)
print(X.shape, y.shape)

(6991, 8) (6991, 1)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol
6122,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30
6370,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64
2879,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81
107,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37
29,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [24]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [25]:
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
model2.fit(X_train_scaled, y_train_scaled)

print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")
# fairly low r2 values means this model does not predict the data accurately,
# but the Linear Regression fits better than other tested models.

Training Data Score: 0.12926546420694507
Testing Data Score: 0.1114296826097334


In [32]:
from sklearn.metrics import mean_squared_error

predictions = model2.predict(X_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)
print(f"MSE: {MSE}")
# better to have higher MSE for test data

MSE: 0.1974916533679382


In [26]:
# from sklearn.linear_model import Lasso

# model2 = Lasso(alpha=0.01).fit(X_train_scaled, y_train_scaled)
# print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.0
Testing Data Score: -0.0033468659986151117


In [27]:
# from sklearn.linear_model import Ridge

# model2 = Ridge(alpha=0.01).fit(X_train_scaled, y_train_scaled)
# print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.12926433415648175
Testing Data Score: 0.1116520625828249


In [28]:
# from sklearn.linear_model import ElasticNet

# model2 = ElasticNet(alpha=0.01).fit(X_train_scaled, y_train_scaled)
# print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.01806581827142828
Testing Data Score: 0.013631563133649216


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [34]:
# Create the GridSearchCV model
from sklearn.svm import SVC
model = SVC(kernel="linear")

# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [40]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\jakan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ................... C=1, gamma=0.0001, score=0.671, total=58.6min
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 58.6min remaining:    0.0s
C:\Users\jakan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] .................. C=1, gamma=0.0001, score=0.661, total=149.0min
[CV] C=1, gamma=0.0001 ...............................................


...

[CV] ................... C=1, gamma=0.0005, score=0.671, total=67.8min
[CV] C=1, gamma=0.0005 ...............................................


C:\Users\jakan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ................... C=1, gamma=0.0005, score=0.661, total=89.6min
[CV] C=1, gamma=0.0005 ...............................................


C:\Users\jakan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


KeyboardInterrupt: 

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
# Calculate the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, 
                            target_names=["CONFIRMED", "CANDIDATE/FALSE POSITIVE"]))

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'your_name.sav'
# joblib.dump(your_model, filename)